In [1]:
from sciterra import Atlas
from sciterra.vectorization import vectorizers

import sys
sys.path.append("/Users/nathanielimel/uci/projects/citesim/src")

from analysis.plot import atlas_to_measurements

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


In [2]:
atlas_dirs = {
    "Physics": "/Users/nathanielimel/uci/projects/citesim/outputs/librarian=S2/vectorizer=SciBERT/center=hafenLowredshiftLymanLimit2017",

    "Linguistics": "/Users/nathanielimel/uci/projects/citesim/outputs/librarian=S2/vectorizer=SciBERT/center=Imeletal2022",

    "Medicine": "/Users/nathanielimel/uci/projects/citesim/outputs/librarian=S2/vectorizer=SciBERT/center=Torres2013",

    "Education": "/Users/nathanielimel/uci/projects/citesim/outputs/librarian=S2/vectorizer=SciBERT/center=Ololube2012",

    "Philosophy": "/Users/nathanielimel/uci/projects/citesim/outputs/librarian=S2/vectorizer=SciBERT/center=Bacon2019",

    "Economics": "/Users/nathanielimel/uci/projects/citesim/outputs/librarian=S2/vectorizer=SciBERT/center=West2003",

    "Materials Science": "/Users/nathanielimel/uci/projects/citesim/outputs/librarian=S2/vectorizer=SciBERT/center=Miele2022",

    "Geology": "/Users/nathanielimel/uci/projects/citesim/outputs/librarian=S2/vectorizer=SciBERT/center=ForeroOrtega2021",

    "Mathematics": "/Users/nathanielimel/uci/projects/citesim/outputs/librarian=S2/vectorizer=SciBERT/center=Andre2018",

}

Set new vectorizer

In [3]:
new_vectorizer = "SBERT"

In [12]:
# Load the source atlas for the field
field = "Physics"
atl = Atlas.load(atlas_dirs[field])
len(atl)

25459

In [5]:
from sciterra import Cartographer
def copy_atlas(atl: Atlas, vectorizer: str, **kwargs) -> Atlas:
    """Create a new atlas containing the publications of an old one, and having a projection with a different vectorizer."""

    atl_new = Atlas(
        publications=list(atl.publications.values()),
    )
    crt = Cartographer(vectorizer=vectorizers[vectorizer](device="mps"))
    atl_new = crt.project(atl_new, **kwargs)
    return atl_new

In [6]:
atl_new: Atlas = copy_atlas(atl, new_vectorizer, batch_size=300)
atl_new.projection

Using device: mps.


embedding documents: 30300it [39:03, 12.93it/s]                           


In [8]:
# save to appropriate dir for analysis
save_dir = atlas_dirs[field].replace("vectorizer=SciBERT", f"vectorizer={new_vectorizer}")
save_dir

'/Users/nathanielimel/uci/projects/citesim/outputs/librarian=S2/vectorizer=SBERT/center=West2003'

In [9]:
# save the atlas
# TODO: create an atlas.copy or from_atlas classmethod
atl_new.center = atl.center
atl_new.history = atl.history
atl_new.save(save_dir)


/Users/nathanielimel/uci/projects/sciterra/src/sciterra/mapping/atlas.py:116: UserWarning: Writing to /Users/nathanielimel/uci/projects/citesim/outputs/librarian=S2/vectorizer=SBERT/center=West2003/publications.pkl.
/Users/nathanielimel/uci/projects/sciterra/src/sciterra/mapping/atlas.py:116: UserWarning: Writing to /Users/nathanielimel/uci/projects/citesim/outputs/librarian=S2/vectorizer=SBERT/center=West2003/projection.pkl.
/Users/nathanielimel/uci/projects/sciterra/src/sciterra/mapping/atlas.py:116: UserWarning: Writing to /Users/nathanielimel/uci/projects/citesim/outputs/librarian=S2/vectorizer=SBERT/center=West2003/bad_ids.pkl.
/Users/nathanielimel/uci/projects/sciterra/src/sciterra/mapping/atlas.py:116: UserWarning: Writing to /Users/nathanielimel/uci/projects/citesim/outputs/librarian=S2/vectorizer=SBERT/center=West2003/history.pkl.
/Users/nathanielimel/uci/projects/sciterra/src/sciterra/mapping/atlas.py:116: UserWarning: Writing to /Users/nathanielimel/uci/projects/citesim/outp

In [10]:
# measure with diff vectorizer


df = atlas_to_measurements(
    atl_new,
    vectorizer=vectorizers[new_vectorizer](device="mps"),
    fields_of_study=[field],
)
df

# Okay, this is tricky; we're going to need to keep the identity of the papers when vectorizing; perhaps just by returning a column of identifiers.

Using device: mps.
Between update 113 and the final update (116) there were 1055 publications added to the Atlas.
Convergence criterion dict_items([('num_pubs_added', 1000), ('kernel_size', 16)]) (=> index 113 out of 117 total updates) yields 19871 ids out of 30203 total ids.
computing cosine similarity for 19871 embeddings with batch size 1000.


100%|██████████| 20/20 [00:07<00:00,  2.84it/s]


Computing ['density', 'edginess'] for 19871 publications.


100%|██████████| 19871/19871 [00:27<00:00, 719.42it/s]

There are 14278 total observations after filtering.



/Users/nathanielimel/uci/projects/citesim/src/analysis/plot.py:213: UserWarning: The center publication is not within the set of converged publications.


,density,edginess,references,citations_per_year,is_center,identifier,year
2,17.581009,0.636703,10,0.100000,False,97bfa3e1ddd14a234096b2b19af36d3f7499030a,2010
3,15.033812,0.727116,41,0.600000,False,1dcf896e45a1ad6911a70fab88c69af5fb41ee16,2010
7,17.166090,0.648202,20,0.000000,False,67a38e92b8cc4f52cfec8a48d41e87731d484c84,2015
8,15.303249,0.626470,93,0.111111,False,a606b2d2625737e8502a98f8895bf7c0a847f14f,2011
9,15.186021,0.675494,25,0.125000,False,caab5b62fbb44255e2a7512d5f35499d75d89837,2004
...,...,...,...,...,...,...,...
19858,16.447217,0.679380,4,0.000000,False,aa8a43e40dbc8c13a1c7934609eee7988588546f,2018
19859,16.315063,0.623282,47,0.315789,False,5b9ec9f9b6b297363632c484094334680af4683c,2001
19861,16.497802,0.698506,14,0.636364,False,d5d0da0ed872f5a8e37b9158c576957377f8943d,2009
19862,16.972609,0.656038,3,0.333333,False,27cd4a98a6828d5489d093ee3e2627560c1d3644,2014


In [11]:
import os

save_fn = "all_data.csv"
save_fn = os.path.join(save_dir, save_fn)

df.to_csv(save_fn, index=False)

In [ ]:
# Now do it for the other settings, and refactor in the other notebook in order to compare

# will we combine fields? If so, should probably only do so at the level of z-scored data.